<a href="https://colab.research.google.com/github/Tahimi/convectionDiffusion/blob/main/animated1DConvDiff_analyticalSolution2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analytical solution of Convection-Diffusion Equation in 1D with Dirichlet BCs

In [1]:
# mounting the drive
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# time discretization

tL = 0.0
delT = 0.2
tR = tL + delT
dt = 0.001
stepsNbr = int(delT/dt)
print('stepsNbr = ', stepsNbr)

stepsNbr =  200


In [3]:
# definig the time instants for the saving of y
import numpy as np

delIdx = int(stepsNbr/4) # define saving frequency
sim_times = np.arange(tL, tR+dt, dt)
save_times = sim_times[range(0, len(sim_times), delIdx)]

closest_idx = lambda val, arr: np.abs(arr - val).argmin()
save_indices = [closest_idx(t, sim_times) for t in save_times]

print('save_times = ', save_times, '\n'
      'save_indices = ', save_indices)

save_times =  [0.   0.05 0.1  0.15 0.2 ] 
save_indices =  [0, 50, 100, 150, 200]


In [4]:
# space discretization
import numpy as np

xL = 0.0
delX = 1.0
xR = xL + delX
nx = 500
dx = delX/nx
x = np.linspace(xL, xR, nx)

print('xL = x[0] = ', x[0], '\n'
      'xR = x[-1] = ', x[-1], '\n'
      'dx = ', dx, '\n'
      'nx = ', nx)

xL = x[0] =  0.0 
xR = x[-1] =  1.0 
dx =  0.002 
nx =  500


In [5]:
# setting diffusion and covection parameters

# diffusion coefficient
alpha = 1

# advection velocity
beta = 5

In [6]:
# computing eigenvalues

n = 70
a = np.empty(n)
N = np.arange(1,n+1,1)
for i in N:
    a[i-1]=(-(i*np.pi/delX)**2)

In [7]:
# definig unkown's distribution

def f(x):
    return np.sin(np.pi*x/delX)

In [8]:
# defining the analytical solution
import scipy.integrate

def u(x, t, beta):
    sum = 0
    for i in N:
        sum = sum+2/xR*scipy.integrate.quad(lambda x:np.exp(-beta/(2*alpha**2)*x)*f(x)* \
                                            np.sin(i*np.pi*x/xR),0,xR)[0]*np.sin(i*np.pi*x/xR)* \
                                            np.exp(-alpha**2*(i*np.pi/xR)**2*t)
    return sum*np.exp(-beta**2/(4*alpha**2)*t)*np.exp(beta/(2*alpha**2)*x)

In [9]:
# Loop over time, calculate y and save it at save_times
import pandas as pd

df = pd.DataFrame(data={'x':x})
tSTR = lambda t: str(round(t,3))
for i, t in enumerate(sim_times):
    y = u(x, t, beta)
    if i in save_indices:
        df[f'y_{tSTR(t)}'] = y

In [10]:
df.head(3)

,x,y_0.0,y_0.05,y_0.1,y_0.15,y_0.2
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.002004,0.006234,0.001081,0.000418,0.000181,0.000080
2,0.004008,0.012512,0.002173,0.000841,0.000363,0.000161


In [11]:
# producing the animation
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Create the figure and axis
fig, ax = plt.subplots()
plt.style.use("ggplot")
plt.close()

# Initialize the line plot
t=0.
line, = ax.plot(x, u(x,t,beta), color='b')
lineRef, = ax.plot(x, u(x,t,beta), color='r', linestyle='dashed')
plotTitle = ax.set_title("t = 0.")

ax.set_xlabel('x')
ax.set_ylabel('y')
offset = 0.1
ax.set_xlim(xL-offset,xR+offset)
ax.set_ylim(0-offset,1.+offset)

# Update function for the animation
def update(time_idx):
    t = tL + time_idx*dt
    y = df[f'y_{tSTR(t)}']
    line.set_ydata(y)
    plotTitle.set_text('t = ' + tSTR(t))
    return line,

# Set up the animation
ani = FuncAnimation(fig, update, frames=save_indices, interval=200, blit=False)

# Save the animation as a GIF
gifFileName = '/content/drive/MyDrive/UFAL/advection_diffusion/FiniteDiference_FipyFiniteVolume/animated1DConvDiffDirichletBCs.gif'
ani.save(gifFileName, writer='pillow')

# Display the animation
HTML(ani.to_jshtml())